In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
WANDB_KEY = user_secrets.get_secret("WANDB_KEY")

In [3]:
!pip install -U accelerate transformers datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 89.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 11.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.3.1
    Uninstalling datasets-3.3.1:
      Successfully uninstalled datasets-3.3.1


In [4]:
!pip uninstall bitsandbytes -y
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.1 MB/s eta 0:00:00:00:0100:01


In [5]:
!pip install wandb

In [6]:
import wandb
wandb.login(key=WANDB_KEY)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: inclinedadarsh (inclinedadarsh-kk-wagh-institute). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model
import torch

In [8]:
# Load the model
dataset = load_dataset('databricks/databricks-dolly-15k')

# Filter rows only with open_qa or general_qa category
dataset = dataset.filter(lambda example: example['category'] == 'open_qa'  or example['category'] == 'general_qa')

# Select only first 100 rows for faster training
dataset = DatasetDict({
    'train': dataset['train'].select(range(100))
})

README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 100
    })
})

In [10]:
def format_example(example):
    text = (
        f"<start_of_turn>user\n",
        f"{example['instruction']}<end_of_turn>\n",
        f"<start_of_turn>assistant\n",
        f"{example['response']}<end_of_turn>"
    )

    return {'text': "".join(text)}

dataset = dataset.map(format_example)

print(dataset['train'][0]['text'])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<start_of_turn>user
Why can camels survive for long without water?<end_of_turn>
<start_of_turn>assistant
Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.<end_of_turn>


In [11]:
model_name = 'google/gemma-2-2b-it'

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [13]:
def tokenize_example(example):
    tokens = tokenizer(
        example['text'],
        padding='max_length',
        truncation=True,
        max_length=1024,
    )

    tokens['labels'] = tokens['input_ids'].copy()

    return tokens

tokenized_dataset = dataset.map(tokenize_example)

print(tokenized_dataset)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})


In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [16]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"":0},
    attn_implementation='eager',
    token=HF_TOKEN
)

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [17]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params / 1e9:.2f}B")
print(f"Trainable Parameters (Full Fine-Tuning): {trainable_params / 1e9:.2f}B")

Total Parameters: 1.60B
Trainable Parameters (Full Fine-Tuning): 0.59B


In [19]:
lora_config = LoraConfig(
    r=2,
    lora_alpha=2,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

In [20]:
lora_model = get_peft_model(model, lora_config)

In [21]:
lora_model.print_trainable_parameters()

trainable params: 399,360 || all params: 2,614,741,248 || trainable%: 0.0153


In [22]:
training_args = TrainingArguments(
    output_dir='./gemma-2-lora-finetuned',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    optim='adamw_bnb_8bit',
    logging_steps=2,
    report_to='wandb',
    save_strategy='epoch',
    remove_unused_columns=False  # Important for LoRA
)

In [23]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset['train']
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
wandb.init(project="gemma-2-finetuning", name="gemma-2b-lora-run1")

In [25]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
2,32.984300
4,34.334400
6,30.508400
8,31.678200
10,28.466600
12,25.336000
14,24.541800
16,21.711000
18,19.984800
20,18.440500


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:1094: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67df19d7-5b11759c5bdf1b584c629e79;a5fa6b24-265b-4e10-9afb-98799a39941a)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Could not find a config file in google/gemma-2-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=25, training_loss=24.75633586883545, metrics={'train_runtime': 200.7598, 'train_samples_per_second': 0.498, 'train_steps_per_second': 0.125, 'total_flos': 1244109157171200.0, 'train_loss': 24.75633586883545, 'epoch': 1.0})